In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import os
import tensorflow as tf
import pickle


In [8]:
#set cloumn width to none so that it will display full comment is the data column
pd.set_option('display.max_colwidth',None)

#setting GPUs
gpus=tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

In [9]:
#importing data 
data=pd.read_csv('toxic.csv')

In [10]:
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,0,0,0,0,0
1,000103f0d9cfb60f,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember what page that's on?",0,0,0,0,0,0


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [12]:
data.sample(5)#return random rows

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
154755,b2e303dc10844d77,"But I am not saying that quantum aetherdynamcis is true. I am just saying what has been documented about it, by sources that are independent from Wikipedia and reliable enough for the University of Pittsburgh to host them on its site.",0,0,0,0,0,0
775,0211c2463faca41a,"""\n for everyone's info (particularly admins), i believe the latest sock-puppet for Igor is . just a """"heads up"""". """,0,0,0,0,0,0
53479,8efc374765133cba,"Congratulations, Tim. Your series of high-quality articles on conductors is some of the best work in the entire Classical Music area of Wikipedia. Just so folks know, here is a list of just a few of your accomplishments in this area alone:\n\nAdrian Boult\nMalcolm Sargent\nThomas Beecham\nJohn Barbirolli\nRichard D'Oyly Carte\n\nCharles Groves\nIsidore Godfrey\nHamilton Clarke\nArthur Sullivan\nGeoffrey Toye",0,0,0,0,0,0
139083,e84e4de45bc7aae4,""":*Indeed, precision is only supposed to come into play when it's a necessity. \n\nRefs\n\nDiscussion\nAny additional comments:\n''The above discussion is preserved as an archive of the proposal. Please do not modify it. Subsequent comments should be made in a new section on this talk page. No further edits should be made to this section.""",0,0,0,0,0,0
21463,389361bc385447fe,"""\nI see your concern, a better article is what I am concerned about too. I've dealt with numerous kinds of text, I know well about text structuring and an aspect drawing my attention (striking) is the linear style of the narrative and the aseptic (surgical?) tone, with one-sided arguments piled up one after the other as a CV (resumée) can be, failing to present the real extent of his actions, not presenting critical approaches to his work or positions, despite his being severely critized by many reputed authors and groups (quite a lenghty criminal record is alleged). Formal rigidity covers up the real content (wasn't that actually what the Pentagon Papers were about? Just wondering...) \n\nI share your concern with the cohesiveness, the paragraph I embedded refers to the Vietnam War context when he was in charge and is intended to clear up what his profile is both at the specific moment and in general as a person, and don't intend to delete his """"medals""""/merits from the article, but offer a critical and balanced approach in the argument (""""completely different subject?"""", well it is directly related to the matter), so the reader can get the whole picture. A domestic issue? Yes (and no), the course of events at home were eventually to prompt the USA pull-out, so both are interacting. I accept suggestions anyway for another collocation or section for the paragraph in question. Thank you for mending, Ellsberg may not be a high-ranking official and needs a fix, there may be a proper word I am not familiar to (mid-ranking official?, I'll look into it). Top-secret, of course they were. The quotes are telling, I cited what the Papers revealed so that the reader gets the picture. My claim of not editing is just a mirror of your claim. Like you, I have a clear position as to the kind of figure and person Kissinger is, but I do not intend to make an article based on my point of view, just to balance it and offer a more realistic picture of the person in question, reality is dirtier, so to say. As I told before, this is not a CV. """,0,0,0,0,0,0


In [13]:
data[data['toxic']==1].sample(5)#gives the random rows with toxic column value 1

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
101325,1e568a93247a6dc3,"Idiot \n\nI didn't vandalise gotye, i left a message on the talk page you tw@t! 81.108.7.13",1,0,1,0,1,0
125970,a1b7c5ea71a4ae7a,My Birthday is tomorrow. I want to get what I want as my birthday not a disgrace made by some internet hacking bitches. User:Factual80man,1,0,1,0,1,0
135802,d6649730f72a5bab,"That's libel right there. Murder is defined as 'unlawful killing of another with malice aforethought ' you stupid wop. Attack pilots have to be trained to be aggressive because if they always fly exactly according to book, they will die on the first day of battle. -",1,0,0,0,0,0
134452,cefb0d2d49e57121,"Haha, screw it. You are all complete and utter jokes, every last one of you (et tu, ZScout). May the Pantone 17-6153 TC — #ffffff — Pantone 18-1662 TC Italian flag live on forever! The perfect visual testament of the lot of you! p ¦",1,0,0,0,0,0
8558,16b8c15eda866ba4,"Calling idiots stupid and retarded doesn't qualify as a personal attack because it's true, and the only way you wouldn't know this is because you're stupid and retarded.",1,0,1,0,1,0


In [14]:
from tensorflow.keras.layers import TextVectorization
#allows us to use text vectorization layer 
# it is a preprocessing layer which maps text features to integer sequences i.e tokanization

In [15]:
#splitting data
x=data['comment_text']
y=data[data.columns[2:]].values #gives array 

In [16]:
#Number of words in our vocabulary
MAX_WORDS=200000;

#instantiated text vectorization layer
vectorizer=TextVectorization(max_tokens=MAX_WORDS,
                             output_sequence_length=1800,
                             output_mode='int')

In [17]:
#We are using TextVectorization to create numerical representation of our text data
#we train vectorizer what our vocabulary is and adapter is going to learn all of the words in our vocabulary 
vectorizer.adapt(x.values)#x.values is of type numpy array

In [18]:
#to check how int is mapped with string input
vectorizer("Hello alll, how are you")[:5]

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([  288, 52674,    73,    20,     7], dtype=int64)>

In [19]:
print(vectorizer.get_vocabulary()[:100])

['', '[UNK]', 'the', 'to', 'of', 'and', 'a', 'you', 'i', 'is', 'that', 'in', 'it', 'for', 'this', 'not', 'on', 'be', 'as', 'have', 'are', 'your', 'with', 'if', 'article', 'was', 'or', 'but', 'page', 'my', 'an', 'from', 'by', 'do', 'at', 'about', 'me', 'so', 'wikipedia', 'can', 'what', 'there', 'all', 'has', 'will', 'talk', 'please', 'would', 'its', 'no', 'one', 'just', 'like', 'they', 'he', 'dont', 'which', 'any', 'been', 'should', 'more', 'we', 'some', 'other', 'who', 'see', 'here', 'also', 'his', 'think', 'im', 'because', 'know', 'how', 'am', 'people', 'why', 'edit', 'articles', 'only', 'out', 'up', 'when', 'were', 'use', 'then', 'may', 'time', 'did', 'them', 'now', 'being', 'their', 'than', 'thanks', 'even', 'get', 'make', 'good', 'had']


In [20]:
#store the vectorized values in vectorized_text named numpy array
vectorized_text=vectorizer(x.values)

In [21]:
vectorized_text
#as our maximum length is 1800 the remaining posistions rather than words filled with 0's

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]], dtype=int64)>

In [22]:
#tensorflow data pipeline
#MCSHBAP - map, cache, shuffle, batch, prefetch
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [23]:
batch_X, batch_y = dataset.as_numpy_iterator().next()

In [24]:
#This gives the values of vectorized text 
batch_X

array([[    51, 162861,     10, ...,      0,      0,      0],
       [ 59500,    441,      9, ...,      0,      0,      0],
       [    48,      2,  11852, ...,      0,      0,      0],
       ...,
       [   171, 124005,     39, ...,      0,      0,      0],
       [    14,     28,      9, ...,      0,      0,      0],
       [   935,      3,     66, ...,      0,      0,      0]], dtype=int64)

In [25]:
#gives the array of dependent features
batch_y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [26]:
#creating training, validation and test partitions
#70 perecent of data for training
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [27]:
print(len(train))
print(len(val))
print(len(test))


6981
1994
997


In [28]:
#this creates a generator so that we can get the next batch using .next()
train_generator = train.as_numpy_iterator()

In [29]:
#every time we run this will get next batch
train_generator.next()

(array([[  136,   178,    14, ...,     0,     0,     0],
        [    2,  4542,     4, ...,     0,     0,     0],
        [   12,   524,    29, ...,     0,     0,     0],
        ...,
        [45421,  8516,  8874, ...,     0,     0,     0],
        [   70,   164,     3, ...,     0,     0,     0],
        [21512,  1394,     6, ...,     0,     0,     0]], dtype=int64),
 array([[0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]], dtype=int64))

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [31]:
def build_model():
    model = Sequential()
    # Embedding layer
    model.add(Embedding(MAX_WORDS+1, 32))#plus one is for unknown word , unknown words have to be seperate embedding
    # Bidirectional LSTM layer
    model.add(Bidirectional(LSTM(32, activation= 'tanh')))
    # Feature extractor dense layers
    model.add(Dense(128, activation= 'relu'))
    model.add(Dense(256, activation= 'relu'))
    model.add(Dense(128, activation= 'relu'))
    # final layer with multiple outputs
    model.add(Dense(6, activation= 'sigmoid'))
    # Compiling the model
    model.compile(loss= 'BinaryCrossentropy', optimizer= 'Adam')
    
    return model

In [32]:
model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [33]:
# Training the model

history = model.fit(train, epochs = 1, validation_data= val)

6981/6981 [==============================] - 10370s 1s/step - loss: 0.0618 - val_loss: 0.0489


In [35]:
history.history
pickle.dump(model,open("model.pkl","wb"))
pickle.dump(history,open("history.pkl","wb"))

In [36]:
# make predictions
input_text = vectorizer('You freaking suck !')

In [37]:
res = model.predict(np.expand_dims(input_text, 0))

1/1 [==============================] - 2s 2s/step


In [38]:
(res>0.5).astype('int')

array([[1, 0, 1, 0, 1, 0]])

In [39]:
#testing on test data 
batch = test.as_numpy_iterator().next()

In [40]:
batch_X, batch_Y = test.as_numpy_iterator().next()

In [41]:
batch_Y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [42]:
(model.predict(batch_X)>0.5).astype('int')

1/1 [==============================] - 0s 338ms/step


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

Model Evaluation

In [43]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [44]:
prec =Precision()
rec =Recall()
acc = CategoricalAccuracy()

In [45]:
for batch in test.as_numpy_iterator():
    X_true, y_true = batch
    yhat = model.predict(X_true)
    
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    prec.update_state(y_true, yhat)
    rec.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 270ms/step


1/1 [==============================] - 0s 315ms/step


1/1 [==============================] - 0s 282ms/step


1/1 [==============================] - 0s 264ms/step


1/1 [==============================] - 0s 293ms/step


1/1 [==============================] - 0s 277ms/step


1/1 [==============================] - 0s 269ms/step


In [46]:
print("Precision: ", prec.result().numpy())
print("Recall: ", rec.result().numpy())
print("Accuracy: ", acc.result().numpy())

Precision:  0.8005865
Recall:  0.7247788
Accuracy:  0.47442326


In [1]:
def predict_comment_hate(comment):
    comment_vectorized = vectorizer([comment])
    results = model.predict(comment_vectorized)
    text=''
    
    for i, col in enumerate(data.columns[2:]):
        text+='{}: {}\n'.format(col, results[0][i] >0.5)
    return text

In [2]:
col ="I will hurt you"

In [3]:
print(predict_comment_hate(col))

NameError: name 'vectorizer' is not defined